<a href="https://colab.research.google.com/github/usamasansi/AiRespository/blob/main/eyesdetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision opencv-python matplotlib


In [2]:
import cv2

def preprocess_image(image_path):
    # Load the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Load Haar Cascade for eye detection
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    eye_regions = []
    for (x, y, w, h) in eyes:
        eye_region = gray[y:y+h, x:x+w]
        eye_resized = cv2.resize(eye_region, (64, 64))
        eye_regions.append(eye_resized)

    return eye_regions


In [3]:
import torch
import torch.nn as nn

class EyeStatusModel(nn.Module):
    def __init__(self):
        super(EyeStatusModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 1)  # Binary classification (open/closed)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid for binary output
        return x


In [6]:
import os
import zipfile
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms

# 1. Unzip the Dataset
zip_path = "/content/sample_data/archive (1).zip"
dataset_path = "/content/data"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

# 2. Custom Dataset Class
class EyeDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = []
        self.labels = []

        for label, folder in enumerate(["close eyes", "open eyes"]):
            folder_path = os.path.join(image_dir, folder)
            for img_file in os.listdir(folder_path):
                self.images.append(os.path.join(folder_path, img_file))
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (64, 64))  # Resize to 64x64
        if self.transform:
            image = self.transform(image)
        else:
            image = torch.tensor(image, dtype=torch.float32).unsqueeze(0) / 255.0  # Normalize
        return image, torch.tensor(label, dtype=torch.long)

# 3. Load Data
train_dir = os.path.join(dataset_path, "data/train")
test_dir = os.path.join(dataset_path, "data/test")

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

train_dataset = EyeDataset(train_dir, transform=train_transform)
test_dataset = EyeDataset(test_dir)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 4. Define the Model
class EyeStatusModel(nn.Module):
    def __init__(self):
        super(EyeStatusModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2),  # 2 classes: open, closed
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

# 5. Train the Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EyeStatusModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")

train_model(model, train_loader, val_loader)

# 6. Prediction Function
def predict_eye_status(image_path, model):
    model.eval()
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (64, 64))
    image = torch.tensor(image, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0
    image = image.to(device)

    with torch.no_grad():
        output = model(image)
        probabilities = torch.softmax(output, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
        openness = confidence.item() * 100
        status = "open" if predicted.item() == 1 else "closed"

    return status, openness

# 7. Test the Model
test_image = os.path.join(test_dir, "/content/sample_data/eyeopen.jpg")  # Replace with a valid image path
status, openness = predict_eye_status(test_image, model)
print(f"Eye Status: {status}, Openness: {openness:.2f}%")

Epoch 1/10, Train Loss: 0.233791609968804, Val Loss: 0.16421118393219614
Epoch 2/10, Train Loss: 0.15544790875013412, Val Loss: 0.15513137785568396
Epoch 3/10, Train Loss: 0.13332907401251512, Val Loss: 0.10121487780271823
Epoch 4/10, Train Loss: 0.115822463313685, Val Loss: 0.09218696032335467
Epoch 5/10, Train Loss: 0.10413209704952917, Val Loss: 0.07475913580372172
Epoch 6/10, Train Loss: 0.09159012475262153, Val Loss: 0.07415791747931194
Epoch 7/10, Train Loss: 0.08286108370786205, Val Loss: 0.06230779396554131
Epoch 8/10, Train Loss: 0.07805794101022823, Val Loss: 0.05701738982798565
Epoch 9/10, Train Loss: 0.07264745277049335, Val Loss: 0.06140029621460892
Epoch 10/10, Train Loss: 0.068283895731048, Val Loss: 0.05434874674505117
Eye Status: closed, Openness: 100.00%


In [7]:
# 7. Test the Model
test_image = os.path.join(test_dir, "/content/sample_data/eyeclose.jpg")  # Replace with a valid image path
status, openness = predict_eye_status(test_image, model)
print(f"Eye Status: {status}, Openness: {openness:.2f}%")

Eye Status: closed, Openness: 57.24%


In [8]:

# 7. Test the Model
test_image = os.path.join(test_dir, "/content/sample_data/syeclosef.jpg")  # Replace with a valid image path
status, openness = predict_eye_status(test_image, model)
print(f"Eye Status: {status}, Openness: {openness:.2f}%")

Eye Status: closed, Openness: 100.00%


In [9]:
# 7. Test the Model
test_image = os.path.join(test_dir, "/content/sample_data/eyeopen.jpg")  # Replace with a valid image path
status, openness = predict_eye_status(test_image, model)
print(f"Eye Status: {status}, Openness: {openness:.2f}%")

Eye Status: closed, Openness: 100.00%
